In [1]:
import numpy as np
import matplotlib.dates
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.ticker import (MultipleLocator)
import pandas as pd
from datetime import datetime

In [2]:
votes = pd.DataFrame()

In [3]:
votescsv = pd.read_csv('battleground-state-changes.csv')

In [4]:
#take first word (ignore "(EV: X)")
votes['state'] = votescsv['state']
for i,row in votes.iterrows():
    row['state'] = row['state'].split(' (')[0]

In [5]:
votes['timestamp'] = pd.to_datetime(votescsv['timestamp'],format='%Y-%m-%d %H:%M:%S.%f')

In [6]:
votes['biden']     = np.where(votescsv['leading_candidate_name']=='Biden',votescsv['leading_candidate_votes'],votescsv['trailing_candidate_votes'])
votes['trump']     = np.where(votescsv['leading_candidate_name']=='Trump',votescsv['leading_candidate_votes'],votescsv['trailing_candidate_votes'])

In [7]:
votes.sort_values(by='timestamp',inplace=True)

In [8]:
thresh = 10 #cut off vote dumps smaller than this
tmin = ''
tmax = '2020-11-07 00:00:00.000'
states = ['Georgia','Pennsylvania'] #['Alaska','Arizona','Georgia','North Carolina','Nevada','Pennsylvania']

In [10]:
for state in states:

    #query dataframe
    stdf  = votes.query(("state == '%s'" % state),inplace=False)
    if(tmin!=''): stdf  = stdf[stdf['timestamp'] >= tmin]
    if(tmin!=''): stdf  = stdf[stdf['timestamp'] <= tmax]

    #main vals
    times = stdf['timestamp'].dt.to_pydatetime()
    biden = stdf['biden'].values
    trump = stdf['trump'].values

    #cumulative fractions (up until second-last val)
    total  = biden[:-1] + trump[:-1]
    fbiden = biden[:-1]/total
    ftrump = trump[:-1]/total
    
    #incoming vote increments
    dtimes =             times[:-1]
    dbiden = biden[1:] - biden[:-1]
    dtrump = trump[1:] - trump[:-1]
    dtotal = dbiden + dtrump
    
    #relative increment (change as percentage of former value)
    rbiden = dbiden / biden[:-1]
    rtrump = dtrump / trump[:-1]
    
    #remove increments larger than threshold
    thresh = 10
    dtimes = dtimes[np.abs(dtotal)>=thresh]
    dbiden = dbiden[np.abs(dtotal)>=thresh]
    dtrump = dtrump[np.abs(dtotal)>=thresh]
    rbiden = rbiden[np.abs(dtotal)>=thresh]
    rtrump = rtrump[np.abs(dtotal)>=thresh]
    fbiden = fbiden[np.abs(dtotal)>=thresh]
    ftrump = ftrump[np.abs(dtotal)>=thresh]
    dtotal = dtotal[np.abs(dtotal)>=thresh]
    
    #fractions of total vote increment
    fdbiden = dbiden/dtotal
    fdtrump = dtrump/dtotal

    #negated biden vals for visualization
    dbiden_neg  = -1*dbiden
    fdbiden_neg = -1*fdbiden
    
    #ratio of obs to exp number of votes in this new batch
    obsbiden = dbiden
    expbiden = fbiden*dtotal #current fraction of votes times number of incoming votes
    obstrump = dtrump
    exptrump = ftrump*dtotal #ditto
    ratbiden = obsbiden/expbiden
    errbiden = np.sqrt(obsbiden)/expbiden
    rattrump = obstrump/exptrump
    errtrump = np.sqrt(obstrump)/exptrump
    
    #plotting
    plt.rcParams["figure.figsize"] = [8,8]
    fig, (top,mid,low,bot) = plt.subplots(4, sharex=True)

    #top: total number of votes for each
    top.plot(times,trump,c='r',linewidth=0.7,label='Trump')
    top.plot(times,biden,c='b',linewidth=0.7,label='Biden')
    top.grid(axis='x')
    top.set_ylabel('total votes')
    top.legend(loc="center right",facecolor='white',framealpha=1,frameon=False)

    #upper middle: visualize numbers of new votes and lead of candidate
    mid.bar(dtimes,dbiden_neg/1000,width=0.007,color='b',zorder=5)
    mid.bar(dtimes,dtrump/1000,width=0.007,color='r',zorder=5)
    mid.plot(times,(biden-trump)/10000,color='b',linewidth=0.5,zorder=3)
    mid.plot(times,(trump-biden)/10000,color='r',linewidth=0.5,zorder=3,label='total vote lead $\div$ 10')
    mid.set_ylabel('new votes ($N\geq$ %s)' % str(thresh))
    mid.set_axisbelow(True)
    if(state=='Pennsylvania'):
        mid.set_ylim(-64,64)
        mid.yaxis.set_major_locator(MultipleLocator(25))
        mid.set_yticklabels(['','50k','25k','0','25k','50k'])
    elif(state=='Georgia'):
        mid.set_ylim(-11,11)
        mid.yaxis.set_major_locator(MultipleLocator(5))
        mid.set_yticklabels(['','10k','5k','0','5k','10k'])
    mid.grid(axis='x') 
    mid.grid(axis='y',linewidth=0.6,linestyle='dashed')
    mid.axhline(0,linewidth=0.6,color='black',zorder=1)
    mid.legend(loc="upper right",facecolor='white',framealpha=1,frameon=False)

    #lower middle: same as above but in percentages
    low.plot(dtimes,-100*fbiden,color='b',linewidth=0.5)
    low.plot(dtimes,100*ftrump,color='r',linewidth=0.5,label='total vote percentage')
    low.bar(dtimes,100*fdbiden_neg,width=0.007,color='b')
    low.bar(dtimes,100*fdtrump,width=0.007,color='r')
    low.scatter(dtimes,100*(fdbiden_neg+fdtrump),color='black',marker="_",s=5,zorder=3)
    low.set_ylabel('new votes ($N\geq$ %s) [%%]' % str(thresh))
    low.set_axisbelow(True)
    low.set_ylim(-101,110)
    low.grid(axis='x') 
    low.grid(axis='y',linewidth=0.6,linestyle='dashed')
    low.axhline(0,linewidth=0.6,color='black',zorder=1)
    low.legend(loc="upper right",ncol=2,facecolor='white',framealpha=1,frameon=False)
    low.set_yticklabels([200,100,50,0,50,100])

    #bottom: compare observed number of votes with that expected for this batch of new votes assuming the current fraction of popularity from the cumulative vote total
    bot.scatter(dtimes,ratbiden,facecolors='none', edgecolors='b',s=2,linewidths=0.5,zorder=3)
    bot.scatter(dtimes,rattrump,facecolors='none', edgecolors='r',s=2,linewidths=0.5,zorder=3)
    bot.errorbar(dtimes,ratbiden,errbiden,c='b',ls='none',elinewidth=0.5,zorder=3)
    bot.errorbar(dtimes,rattrump,errtrump,c='r',ls='none',elinewidth=0.5,zorder=3)
    bot.set_ylim(-0.1,2.1)
    bot.set_axisbelow(True)
    bot.grid(axis='x')
    bot.grid(axis='y',linewidth=0.6,linestyle='dashed')
    bot.axhline(1,linewidth=0.6,color='black',zorder=1)
    bot.set_ylabel('$N_{obs} \pm \sqrt{N_{obs}} \div N_{exp,bulk}$')
    bot.xaxis_date()
    bot.set_xlabel('date, time')

    plt.xticks(rotation=45)
    plt.tight_layout()
    
    fig.savefig('%s.pdf' % state)
    plt.close()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in sqrt
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in sqrt
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:82: UserWarning: FixedFormatter should only be used together with FixedLocator
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:101: UserWarning: FixedFormatter should only be used together with FixedLocator
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in sqrt
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in sqrt
/Library/Frameworks/Pytho